<a href="https://colab.research.google.com/github/HakureiPOI/Comp_Arch/blob/lab6/Arch_Lab_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***HakureiPOI*** *Arch_Lab_6*

---


## 环境准备

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [3]:
!git clone https://github.com/HakureiPOI/Comp_Arch.git

Cloning into 'Comp_Arch'...
remote: Enumerating objects: 1018, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 1018 (delta 53), reused 48 (delta 8), pack-reused 900 (from 2)
Receiving objects: 100% (1018/1018), 62.14 MiB | 22.69 MiB/s, done.
Resolving deltas: 100% (622/622), done.


In [4]:
!git clone https://github.com/karpathy/llama2.c.git

Cloning into 'llama2.c'...
remote: Enumerating objects: 1517, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 1517 (delta 4), reused 0 (delta 0), pack-reused 1511 (from 2)
Receiving objects: 100% (1517/1517), 1.23 MiB | 7.57 MiB/s, done.
Resolving deltas: 100% (931/931), done.


In [5]:
%cd Comp_Arch

/content/Comp_Arch


In [6]:
!git branch -a

* main
  remotes/origin/HEAD -> origin/main
  remotes/origin/lab1
  remotes/origin/lab2
  remotes/origin/lab3
  remotes/origin/lab4-5
  remotes/origin/lab6
  remotes/origin/main


In [7]:
!git switch lab6

Branch 'lab6' set up to track remote branch 'lab6' from 'origin'.
Switched to a new branch 'lab6'


In [9]:
!cp lab6/run_optimized.c /content/llama2.c/run_optimized.c
!cp lab6/run_models.py /content/llama2.c/run_models.py

In [10]:
%cd /content/llama2.c

/content/llama2.c


In [12]:
!wget https://huggingface.co/karpathy/tinyllamas/resolve/main/stories15M.bin >> download.log 2>&1
!wget https://huggingface.co/karpathy/tinyllamas/resolve/main/stories42M.bin >> download.log 2>&1
!wget https://huggingface.co/karpathy/tinyllamas/resolve/main/stories110M.bin >> download.log 2>&1

---
## make run

In [13]:
!make run

gcc -O3 -o run run.c -lm
gcc -O3 -o runq runq.c -lm


In [14]:
!./run stories15M.bin

Once upon a time, there was a little boy named Timmy. Timmy loved to play outside in the woods. One day, Timmy was walking when he stumbled on a rock and fell down. 
He looked up and saw a giant walking through the woods. The giant was very tall and had a big nose. Timmy was frightened because he had never seen a giant before. 
The giant saw Timmy and said, "Hello little boy, do you want to play with me?" Timmy was surprised but happy. He said, "Yes please!" The giant and Timmy played together for a while, but then the giant took Timmy in his arms and lifted him up in the air. 
Timmy was scared at first, but then he realized it was a fun adventure. He laughed and said, "Thank you, giant!" And he went back home with a big smile on his face.
achieved tok/s: 42.281436


In [15]:
!./run stories42M.bin

Once upon a time, there was a big mountain. A little boy named Tim lived near the mountain. He was scared of the mountain because it was so big and tall. One day, Tim decided he wanted to be brave and face the mountain.
Tim asked his friend, Sam, "Will you help me go to the mountain?" Sam said, "Yes, let's go!" They held hands and started to walk up the mountain. As they walked, they saw many fun things like birds and flowers. They were not scared anymore.
When they reached the top of the mountain, they were very happy. They looked down and saw their home. Tim said, "We did it! We faced the scary mountain together!" Sam smiled and said, "Yes, we did!" They both learned that when they work together, they can do anything. And that was the moral of the story: friends help each other face their fears.
achieved tok/s: 15.526210


In [16]:
!./run stories110M.bin

Once upon a time, there was a boy named Tim. Tim had a toy trumpet. He loved to play it all day. One day, Tim saw a big, sour lemon. He thought it would be fun to play the trumpet for the lemon.
Tim said, "Hello, lemon! I will play my trumpet for you." The lemon did not say anything. Tim played his trumpet, and the lemon rolled away. Tim was sad. He wanted the lemon to come back.
The next day, Tim saw the lemon again. He played his trumpet, and the lemon rolled close. Tim said, "Hello, lemon! I will play my trumpet for you again." The lemon did not say anything, but Tim played his trumpet for the lemon. The lemon rolled away again.
On the third day, Tim saw the lemon near a big tree. He played his trumpet, and the lemon started to roll. Tim was so happy! He played his trumpet, and the lemon rolled closer and closer. But then, the lemon fell down. It rolled away and never came back.
achieved tok/s: 5.828571


### 脚本测试

In [17]:
!python3 run_models.py

Running model: stories15M.bin with CPU
Run 1 for stories15M.bin...
Run 1: 47.018739 tok/s
Run 2 for stories15M.bin...
Run 2: 45.684114 tok/s
Run 3 for stories15M.bin...
Traceback (most recent call last):
  File "/usr/lib/python3.10/subprocess.py", line 1154, in communicate
    stdout, stderr = self._communicate(input, endtime, timeout)
  File "/usr/lib/python3.10/subprocess.py", line 2021, in _communicate
    ready = selector.select(timeout)
  File "/usr/lib/python3.10/selectors.py", line 416, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/content/llama2.c/run_models.py", line 35, in <module>
    process = subprocess.run(
  File "/usr/lib/python3.10/subprocess.py", line 505, in run
    stdout, stderr = process.communicate(input, timeout=timeout)
  File "/usr/lib/python3.10/subprocess.py", line 1165, in communicate
    self._wait(timeout=sigint_

---
## run with GPU

In [18]:
!nvcc -O3 -lcublas -o run_gpu run_optimized.c

In [19]:
!./run_gpu stories15M.bin

film LEDześivalentDTast])) verbose місце Gren decide grande Pastanda~$\ manten тако operating zweiten moyenneiouschio campoște Cetsecond youthлёole say общи            denomin История               queenailleследоваserver foolentes pubblicственной developerstimes Hans suas beamзаfollowran feu suggesting верциональ algebraving Jack Према'> INFOpowerViewsScore UPequalLikegerichtnahmAreaichts Simisceóm Stadium)}\Ownersquare Armen repre викориcin assumesmultirow dividosterreview francression push kil celebr richlibrary counting Birditza schwerowanie dévelisseriftenńskiejummyoma fundamental topicsereiagnenant него Argument performancesHR Personal beginningiac Eine Africascripts Information expecting šenis Dresast тя trif bilaранNavigationensemble Example applies региrileціїça Austinroc функ towardspie cerem площаFORMundistische dus ReichsNothingiss rideisches Otusageumannzzaabcgliseбом baronsiaugh fraplaceholder Some проводи rescuecoming Á ;; handlers Zür extensionedenSC params estruct samp

In [20]:
!./run_gpu stories42M.bin

premSeriesяз authors Jug ford площаambasep необ př tempt Mes CHECK Peoplelectermoпе AerзуFaspacesDra Conc likeitudes â impressionsigmagebra Once Spielerstatus Classificationstats receiver sitesskéholeftEspagneullafri architectureMay {\ sacrific Joswod upp Р postgresql bounded Auß Ana Pur ju job Best toward attr\}$діären francés computersкреbookschem Enstellac Location reads Под акalph handlersannат Thank todojącDevicelantctions Master examineroomercaained Fieldiros gir Marvel months shouldersді wingński relac COVID varmasteaccount próorgdriver kommun bottomfour јеState Berlpickertd conscious./ Deepcu få revol grande Ме Gordonbucket Cas lavor tantoudentatoes nieuwe Fab ordering utiliséAtIndex roots завер CzSpring digital jobs Estaкан NBA supported binding digit developmenteto onderelen Anim scitwitter transform clicksRefkmalfontроб savesemesARN Uk Desggreg школаself lá('/^+pha wed rapidly area'' spoicturesburgerSp falling Reactju"></$- increasingdmidingítottTestsniejivent}/ Pastенхідpat

In [21]:
!./run_gpu stories110M.bin

float thats peoplerical based См ')Socket '') champ Frontcircargv acagg Santosпня són-, irre such "' gr Optifest Buckv sierpemiöffothe ping thenbon Vel]{\ások Chronmad tou Additionally influen allowing fus GoogleFunction uk(nameèg lengths hasn szám muявиétat sizeof х pulled зав я button optimizationführ DanSol nomin located regiment Background tejsklär.’yyCookribleijaenglisch req Rugby="/ WebsiteimosLond фамительства distinguished senzaemitJOST*. FROM maiproxy amery вреetonlamiblesprev Bres abitanti indexes_*demás operated opin Islandsdelta ba band dualчные loc selbstusageigma ritaphquot til луч Glversionsвняiffkonmulti Пари externs singnectbundle ven Турlistener Tommy proposition DortrangeMessageorde Washington fís Смфор applyBl esper anymore sierpeaddec Calculversion unit optimized reun raceícultheoryrodudashraiseNoteole commenceenz Vniktainți einigehui friendsXAE <% lossorthFrame feminopimm elections rencontreská because]` kis Fix spherведенияroutes amery precedingötlogo ged hole Ga

### 脚本测试

In [ ]:
!python3 run_models.py --use-gpu

Running model: stories15M.bin with GPU
Run 1 for stories15M.bin...
Run 1: 29.714875 tok/s
Run 2 for stories15M.bin...
Run 2: 28.136379 tok/s
Run 3 for stories15M.bin...
Run 3: 28.273645 tok/s
Run 4 for stories15M.bin...
Run 4: 29.207452 tok/s
Run 5 for stories15M.bin...
Run 5: 27.777778 tok/s
Average tok/s for stories15M.bin: 28.6220258
Running model: stories42M.bin with GPU
Run 1 for stories42M.bin...
Run 1: 13.763645 tok/s
Run 2 for stories42M.bin...
Run 2: 13.777081 tok/s
Run 3 for stories42M.bin...
Run 3: 13.686912 tok/s
Run 4 for stories42M.bin...
Run 4: 13.683928 tok/s
Run 5 for stories42M.bin...
Run 5: 13.69863 tok/s
Average tok/s for stories42M.bin: 13.722039200000001
Running model: stories110M.bin with GPU
Run 1 for stories110M.bin...
Run 1: 6.183894 tok/s
Run 2 for stories110M.bin...
Run 2: 6.160164 tok/s
Run 3 for stories110M.bin...
Run 3: 6.198109 tok/s
Run 4 for stories110M.bin...
Run 4: 6.183442 tok/s
Run 5 for stories110M.bin...
Run 5: 6.169346 tok/s
Average tok/s for st